In [20]:
#download metadata from dryad repository using the dryad API and save it as a csv file 
#against https://datadryad.org/api/v2/docs/#/root/get_search

import requests
import json
import pandas as pd

#iterate over each page of the search results and save the metadata in a list
def get_dryad_metadata():
    dryad_metadata = []
    page = 1
    while True:
        url = "https://datadryad.org/api/v2/search?q=Neuroscience&page=" + str(page)
        headers = {'Content-Type': 'application/json'}
        response = requests.get(url, headers=headers)
        response_json = response.json()
        if len(response_json['_embedded']["stash:datasets"]) == 0:
            break
        dryad_metadata.extend(response_json['_embedded']["stash:datasets"])
        page += 1
    return dryad_metadata

data = get_dryad_metadata()

In [28]:

data_df = pd.DataFrame(data)
#filtering out collumns with obsolete data
data_df.drop(['_links','versionNumber', 'versionChanges'], axis=1, inplace=True)

In [29]:
#filter out html tags and new lines from methods and abstract column
for col in ['methods', 'abstract', 'usageNotes']:
    data_df[col] = data_df[col].str.replace('<[^<]+?>', '')
    data_df[col] = data_df[col].str.replace('\n', ' ')
    data_df[col] = data_df[col].str.replace('\r', ' ')
    


C:\Users\Luki\AppData\Local\Temp\ipykernel_16112\2889919398.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df[col] = data_df[col].str.replace('<[^<]+?>', '')


In [30]:
print(data_df.head())

                    identifier     id   storageSize relatedPublicationISSN  \
0  doi:10.5061/dryad.mw6m905wb  65201        269166              0028-3878   
1  doi:10.5061/dryad.vq83bk3s8  67999    6461489656              2050-084X   
2  doi:10.5061/dryad.83bk3j9nr  45368        131236              1045-2249   
3  doi:10.5061/dryad.95x69p8gr  46633        266227                    NaN   
4  doi:10.5061/dryad.6wwpzgmx4  59348  238930232814              0306-4522   

                                               title  \
0  Kidney function, kidney function decline, and ...   
1  Spatially displaced excitation contributes to ...   
2  Data from: Volatile fatty acid and aldehyde ab...   
3  A comparison between implicit and explicit sel...   
4  Evoking the N400 Event-Related Potential (ERP)...   

                                             authors  \
0  [{'firstName': 'Hong', 'lastName': 'Xu', 'emai...   
1  [{'firstName': 'Jennifer', 'lastName': 'Ding',...   
2  [{'firstName': 'Stephan

In [31]:
#save metadata to csv file, making unexisting folders
import os
if os.path.exists('../data/Dryad') == False:
    os.makedirs('../data/Dryad')
data_df.to_csv('../data/Dryad/dryad_metadata.csv', index=False)